In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import os

from os.path import dirname



root_path = dirname(os.getcwd()) + "/HGNN_NA"

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI20_RequestForPayment"

In [3]:
categorical_columns = ['org:group', 'resource country', 'org:resource', 'oranization country',
       'org:role', 'Activity', 'impact', 'product', "org:resource:role"]
real_value_columns = ["time:timestamp"]

In [4]:
tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")
tab_all.head()

,org:resource,Activity,time:timestamp,org:role,case:Project,case:Task,CaseID,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber,org:resource:role
0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,0.000000,EMPLOYEE,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215,Role 1
1,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,3.737670,SUPERVISOR,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215,Role 1
2,STAFF MEMBER,Request For Payment REJECTED by MISSING,11.499982,MISSING,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215,Role 1
3,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,15.337479,EMPLOYEE,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215,Role 1
4,STAFF MEMBER,Request For Payment APPROVED by PRE_APPROVER,15.337486,PRE_APPROVER,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,3.536204,UNKNOWN,request for payment number 148215,Role 1


In [5]:
tab_all.columns

Index(['org:resource', 'Activity', 'time:timestamp', 'org:role',
       'case:Project', 'case:Task', 'CaseID', 'case:OrganizationalEntity',
       'case:Cost Type', 'case:RequestedAmount', 'case:Activity',
       'case:RfpNumber', 'org:resource:role'],
      dtype='object')

In [6]:
categorical_columns = ['org:resource', 'Activity', 'org:role',
      'case:Project', 'case:Task', 
       'case:OrganizationalEntity', 'case:Cost Type',
       'case:Activity', 'case:RfpNumber', "org:resource:role"]
real_value_columns = ["time:timestamp", "case:RequestedAmount"]

{'org:resource': ['STAFF MEMBER', 'SYSTEM'],
 'Activity': ['Request For Payment SUBMITTED by EMPLOYEE',
  'Request For Payment FINAL_APPROVED by SUPERVISOR',
  'Request For Payment REJECTED by MISSING',
  'Request For Payment APPROVED by PRE_APPROVER',
  'Request Payment',
  'Payment Handled',
  'Request For Payment REJECTED by SUPERVISOR',
  'Request For Payment REJECTED by EMPLOYEE',
  'Request For Payment APPROVED by SUPERVISOR',
  'Request For Payment FINAL_APPROVED by DIRECTOR',
  'Request For Payment REJECTED by PRE_APPROVER',
  'Request For Payment SAVED by EMPLOYEE',
  'Request For Payment REJECTED by ADMINISTRATION',
  'Request For Payment APPROVED by ADMINISTRATION',
  'Request For Payment APPROVED by BUDGET OWNER',
  'Request For Payment REJECTED by BUDGET OWNER',
  'Request For Payment FOR_APPROVAL by ADMINISTRATION',
  'Request For Payment FINAL_APPROVED by BUDGET OWNER',
  'Request For Payment FOR_APPROVAL by SUPERVISOR'],
 'org:role': ['EMPLOYEE',
  'SUPERVISOR',
  'MISS

In [7]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [8]:
with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_FINAL.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_FINAL.pkl", "rb") as f:
    X_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_FINAL.pkl", "rb") as f:
    X_test = pickle.load(f)

In [9]:

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToUndirected, NormalizeFeatures

transform = ToUndirected()

with torch.no_grad():
        for i in range(len(X_train)):
                X_train[i] = transform(X_train[i])
        for i in range(len(X_valid)):
                X_valid[i] = transform(X_valid[i])
        for i in range(len(X_test)):
                X_test[i] = transform(X_test[i])
    






In [10]:
edge_types = set()
node_types = set()
for i in range(len(X_train)):
    n, edge_type = X_train[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_valid)):
    n, edge_type = X_valid[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_test)):
    n, edge_type = X_test[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)



In [11]:
node_types = list(node_types)
edge_types = list(edge_types)

In [12]:
node_types

['org:resource',
 'time:timestamp',
 'case:Cost Type',
 'org:role',
 'Activity',
 'case:OrganizationalEntity',
 'case:RequestedAmount',
 'case:Project',
 'case:RfpNumber',
 'case:Activity',
 'case:Task']

In [13]:
edge_types

[('Activity', 'related_to', 'case:Activity'),
 ('Activity', 'related_to', 'case:Cost Type'),
 ('org:role', 'related_to', 'org:role'),
 ('Activity', 'related_to', 'case:Task'),
 ('case:OrganizationalEntity', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:RequestedAmount'),
 ('Activity', 'related_to', 'org:resource'),
 ('case:Project', 'rev_related_to', 'Activity'),
 ('case:RequestedAmount', 'rev_related_to', 'Activity'),
 ('org:role', 'rev_related_to', 'Activity'),
 ('time:timestamp', 'related_to', 'time:timestamp'),
 ('Activity', 'related_to', 'case:Project'),
 ('case:Activity', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:RfpNumber'),
 ('org:resource', 'rev_related_to', 'Activity'),
 ('time:timestamp', 'rev_related_to', 'Activity'),
 ('Activity', 'related_to', 'case:OrganizationalEntity'),
 ('case:Cost Type', 'rev_related_to', 'Activity'),
 ('Activity', 'followed_by', 'Activity'),
 ('case:RfpNumber', 'rev_related_to', 'Activity'),
 ('org:resource'

In [12]:
def get_weights(load, key):
    weights = []
    
    cl_train = [0 for _ in tab_all[key].unique()]
    
    print(cl_train)
    
    for i,x in enumerate(load):

        
        classes = x.y[key]

        # print(classes)
        
        for c in list(classes):
            try:
                cl_train[c] +=1
            except KeyError:
                cl_train[c] = 1
    s = sum(cl_train)
    
    print(cl_train)
    
    weights = [s/x if x > 0 else 0 for x in cl_train]

    # weights = [0.7,0.7,1,0.7,0.7,0.7,0.7,0.7,0.7,0.7]
    weights = torch.tensor(weights, device=device)
    print(weights)
    return weights
    

In [14]:
edge_features_dims = {}

for i in range(len(X_train)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_valid)):
    for k in X_valid[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_valid[i].edge_attr_dict[k].shape[1]
for i in range(len(X_test)):
    for k in X_test[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_test[i].edge_attr_dict[k].shape[1]

edge_features_dims

{('org:resource', 'related_to', 'org:resource'): 2,
 ('Activity', 'followed_by', 'Activity'): 2,
 ('time:timestamp', 'related_to', 'time:timestamp'): 2,
 ('org:role', 'related_to', 'org:role'): 2,
 ('org:resource:role', 'related_to', 'org:resource:role'): 2}

In [15]:
act_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "Activity")
# res_roles_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "org:resource:role")


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3348, 228, 897, 42, 14, 0, 26, 2778, 1, 0, 31, 19, 534, 66, 2, 82, 1, 341, 3348]
tensor([3.5119e+00, 5.1570e+01, 1.3108e+01, 2.7995e+02, 8.3986e+02, 0.0000e+00,
        4.5223e+02, 4.2325e+00, 1.1758e+04, 0.0000e+00, 3.7929e+02, 6.1884e+02,
        2.2019e+01, 1.7815e+02, 5.8790e+03, 1.4339e+02, 1.1758e+04, 3.4481e+01,
        3.5119e+00], device='cuda:0')


In [16]:
from data import utils

one_hot_resource = utils.get_one_hot_encoder(tab_all, "org:resource")
one_hot_resource_role = utils.get_one_hot_encoder(tab_all, "org:resource:role")
resources = tab_all["org:resource"]
resources_role = tab_all["org:resource:role"]
map_resource_to_role = {}
for i in range(len(resources)):
    map_resource_to_role[resources[i]] = resources_role[i]

map_resource_to_role

{'STAFF MEMBER': 'Role 1', 'SYSTEM': 'Role 2'}

In [17]:
map_resource_index = {}
for x in tab_all["org:resource"].unique():
    map_resource_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource, np.array([x])))).item()
map_resource_index

{'STAFF MEMBER': 0, 'SYSTEM': 1}

In [18]:
map_resource_role_index = {}
for x in tab_all["org:resource:role"].unique():
    map_resource_role_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource_role, np.array([x])))).item()
map_resource_role_index

{'Role 1': 0, 'Role 2': 1}

In [19]:
map_resource_to_role = { map_resource_index[k] : map_resource_role_index[v] for k,v in map_resource_to_role.items()}
map_resource_to_role

{0: 0, 1: 1}

## Hyperopt

In [20]:
from ax.service.managed_loop import optimize

In [21]:
from torch_geometric.nn import (
    HeteroConv,
    global_mean_pool,
    GATv2Conv
)
from torch.nn import (
    ModuleList,
    Module,
    Linear
  )
from typing_extensions import Self

In [22]:
class HGNN(Module):

    def __init__(self, output_cat, output_real,nodes_relations, relations_with_features, parameters) -> Self:  # type: ignore
        super().__init__()

        # List of convolutional layers
        
        hid = parameters["hid"]
        layers = parameters["layers"]
        aggregation = parameters["aggregation"]
        n_heads = parameters["heads"]
        
        self.output_cat = output_cat
        self.output_real = output_real
        
        self.convs = ModuleList()
        for _ in range(layers):
            conv = HeteroConv(
                {
                    relation: (
                        GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid, heads=n_heads, concat=False)
                        if relation not in relations_with_features
                        else GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid,heads=n_heads, edge_dim=relations_with_features[relation], concat=False)
                    )
                    for relation in nodes_relations
                },
                aggr=aggregation,
            )

            self.convs.append(conv)

        self.FC = {}
        
        for k in output_cat:
            self.FC[k] = Linear(hid, output_cat[k], device=device)
        for k in output_real:
            self.FC[k] = Linear(hid, 1, device=device)
        
    
        
        

    def forward(self, batch):

        for i in range(len(self.convs)):
            batch.x_dict = self.convs[i]( 
                batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
            )

            batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}


        output = {}
        
        for k in self.output_cat:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k])
        for k in self.output_real:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k]).reshape(1,-1)[0]
            

        return output

In [23]:
from torcheval.metrics.functional import multiclass_accuracy, multiclass_f1_score

In [24]:
import torch.nn as nn

In [25]:
import time

In [26]:
from copy import deepcopy


def train_hgnn(config, output_cat, output_real, epochs=20):
    print(config)

    net = HGNN(
        parameters=config,
        output_cat=output_cat,
        output_real=output_real,
        nodes_relations=edge_types,
        relations_with_features=edge_features_dims,
    )
    net = net.to(device)

    losses = {}

    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()

    train_loader = DataLoader(X_train, batch_size=config["batch_size"], shuffle=True)
    valid_loader = DataLoader(X_valid, batch_size=config["batch_size"], shuffle=True)

    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    best_model = None
    best_loss = 0
    patience = 5
    pat_count = 0

    torch.cuda.empty_cache()

    for epoch in range(0, epochs):
        start_time = time.time()

        print(f"Epoch: {epoch}\n")

        net.train()
        for _, x in enumerate(train_loader):
            x = x.to(device)

            labels = x.y

            optimizer.zero_grad()
            outputs = net(x)

            losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

            total_loss = 0
            for k in losses_step:
                total_loss += losses_step[k]

            total_loss.backward()
            optimizer.step()

        predictions_categorical = {k: [] for k in output_cat}
        target_categorical = {k: [] for k in output_cat}

        avg_MAE = {k: [] for k in output_real}

        running_total_loss = []

        net.eval()
        with torch.no_grad():
            for i, x in enumerate(valid_loader):
                x = x.to(device)

                labels = x.y

                outputs = net(x)

                losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

                running_total_loss.append(sum(list(losses_step.values())))

                for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])

                for k in output_real:
                    avg_MAE[k].append(losses_step[k])

        for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])

        macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )

        accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }

        avg_MAE = {k: sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}

        val_loss = sum(running_total_loss) / len(running_total_loss)

        print(f"\nVALIDATION")
        for k in accuracy:
            (
                print("{}: acc {:.4f}".format(k, accuracy[k]))
                if k != "Activity"
                else print(
                    "{}: acc {:.4f} macroF1 {:.4f}".format(
                        k, accuracy[k], macro_f1_activity.item()
                    )
                )
            )
        for k in avg_MAE:
            print("{}: MAE {:.4f}".format(k, avg_MAE[k]))
        print("TOTAL_LOSS: {:.4f}".format(val_loss))
        print("epoch time {}s\n".format(time.time() - start_time))

        if epoch == 0:
            best_model = deepcopy(net)
            best_loss = val_loss
        else:
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = deepcopy(net)
                pat_count = 0
                print("new best model found")
            if pat_count == patience:
                print(
                    "Validation performance didn't improve for {} epochs. Training stops.".format(
                        pat_count
                    )
                )
                return best_model
        pat_count += 1

    return best_model

In [27]:
def test_hgnn(net, output_cat, output_real):
    test_loader = DataLoader(X_train, batch_size=128, shuffle=False)
    
    losses = {}
    
    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()
    
    
    
    
    
    predictions_categorical = {k: [] for k in output_cat}
    target_categorical = {k: [] for k in output_cat}

    avg_MAE = {k : [] for k in output_real}
    
    total_loss = []
        
    net.eval()
    with torch.no_grad():
        for i, x in enumerate(test_loader):
            x = x.to(device)
            
            labels = x.y
            
            outputs = net(x)
            
     
            losses_step = {k: losses[k](outputs[k], labels[k]).item() for k in losses}
            total_loss.append(sum(list(losses_step.values())))
            
            for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])
            
            
            for k in output_real:
                    avg_MAE[k].append(losses_step[k])
                    
    for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])
               
            
    macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )
            
    accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }
    
    avg_MAE = {k : sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}
    
    resource_to_role_acc = multiclass_accuracy(
        torch.tensor([map_resource_to_role[x.item()] for x in predictions_categorical["org:resource"]], device=device),
        target_categorical["org:resource:role"],
        num_classes=output_cat["org:resource:role"]
    )
    
    Average_total_loss = sum(total_loss) / len(total_loss)
    
    res = {f"{k}_acc" : accuracy[k].item() for k in accuracy} | {"Resource_to_role_acc" : resource_to_role_acc.item()} | {"MacroF1Act" : macro_f1_activity.item()} | {f"{k}_mae" : avg_MAE[k] for k in avg_MAE} | {"AVG_total_loss" : Average_total_loss} 
    
    #print(res)
    
    return res

In [28]:
list_unique = {k : list(tab_all[k].unique()) for k in categorical_columns}
list_unique

{'org:resource': ['STAFF MEMBER', 'SYSTEM'],
 'Activity': ['Request For Payment SUBMITTED by EMPLOYEE',
  'Request For Payment FINAL_APPROVED by SUPERVISOR',
  'Request For Payment REJECTED by MISSING',
  'Request For Payment APPROVED by PRE_APPROVER',
  'Request Payment',
  'Payment Handled',
  'Request For Payment REJECTED by SUPERVISOR',
  'Request For Payment REJECTED by EMPLOYEE',
  'Request For Payment APPROVED by SUPERVISOR',
  'Request For Payment FINAL_APPROVED by DIRECTOR',
  'Request For Payment REJECTED by PRE_APPROVER',
  'Request For Payment SAVED by EMPLOYEE',
  'Request For Payment REJECTED by ADMINISTRATION',
  'Request For Payment APPROVED by ADMINISTRATION',
  'Request For Payment APPROVED by BUDGET OWNER',
  'Request For Payment REJECTED by BUDGET OWNER',
  'Request For Payment FOR_APPROVAL by ADMINISTRATION',
  'Request For Payment FINAL_APPROVED by BUDGET OWNER',
  'Request For Payment FOR_APPROVAL by SUPERVISOR'],
 'org:role': ['EMPLOYEE',
  'SUPERVISOR',
  'MISS

In [29]:
outputcat = {k : len(list_unique[k]) if len(list_unique[k]) > 1 else None for k in list_unique}
outputcat = {k : v for k,v in outputcat.items() if v != None}
outputreal = real_value_columns
print(outputcat)
print(outputreal)

{'org:resource': 2, 'Activity': 19, 'org:role': 8, 'case:Project': 79, 'case:Task': 597, 'case:OrganizationalEntity': 36, 'case:Activity': 6, 'case:RfpNumber': 6322, 'org:resource:role': 2}
['time:timestamp', 'case:RequestedAmount']


In [30]:
def train_evaluate(config):
    trained_net = train_hgnn(config, output_cat=outputcat, output_real=outputreal, epochs = 20)
    return test_hgnn(trained_net, output_cat=outputcat, output_real=outputreal)

In [36]:
config = {'lr': 0.0001865322718407359, 'batch_size': 1024, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}

In [37]:
train_evaluate(config)

{'lr': 0.0001865322718407359, 'batch_size': 1024, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0562 macroF1 0.0082
org:role: acc 0.1008
case:Project: acc 0.1537
case:Task: acc 0.0009
case:OrganizationalEntity: acc 0.0903
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0000
time:timestamp: MAE 15.5245
case:RequestedAmount: MAE 2.4729
TOTAL_LOSS: 49.4724
epoch time 14.107543230056763s

Epoch: 1




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0187 macroF1 0.0038
org:role: acc 0.3723
case:Project: acc 0.1537
case:Task: acc 0.0009
case:OrganizationalEntity: acc 0.0903
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0000
time:timestamp: MAE 14.0935
case:RequestedAmount: MAE 1.5798
TOTAL_LOSS: 45.0189
epoch time 13.19575548171997s

new best model found
Epoch: 2



KeyboardInterrupt: 

In [31]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "hid", "type": "choice", "values": [128], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "layers", "type": "choice", "values": [2, 3, 4, 5], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "layers", "type": "choice", "values": [2], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "lr", "type": "range", "bounds": [1e-4, 1e-1], "value_type": "float", "log_scale": True},
        {"name": "batch_size", "type": "choice", "values": [128, 256, 512], "value_type": "int", "is_ordered" : True,"sort_values":False}, 
        {"name": "heads", "type": "choice", "values": [1,2], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "heads", "type": "choice", "values": [1], "value_type": "int", "is_ordered" : True,"sort_values":False},
        
        #{"name": "aggregation", "type" : "choice", "values" :["sum", "mean", "max"], "value_type" : "str"}
        {"name": "aggregation", "type" : "choice", "values" :["max"], "value_type" : "str"},
     
    ],
  
    evaluation_function=train_evaluate,
    objective_name='AVG_total_loss',
    arms_per_trial=1,
    minimize = True,
    random_seed = 123,
    total_trials = 20
)

print(best_parameters)
means, covariances = values
print(means)
print(experiment)

[INFO 01-13 17:00:51] ax.service.utils.instantiation: Choice parameter hid contains only one value, converting to a fixed parameter instead.
[INFO 01-13 17:00:51] ax.service.utils.instantiation: Choice parameter layers contains only one value, converting to a fixed parameter instead.
[INFO 01-13 17:00:51] ax.service.utils.instantiation: Choice parameter aggregation contains only one value, converting to a fixed parameter instead.
[INFO 01-13 17:00:51] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='hid', parameter_type=INT, value=128), FixedParameter(name='layers', parameter_type=INT, value=2), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.0001, 0.1], log_scale=True), ChoiceParameter(name='batch_size', parameter_type=INT, values=[128, 256, 512], is_ordered=True, sort_values=False), ChoiceParameter(name='heads', parameter_type=INT, values=[1, 2], is_ordered=True, sort_values=False), FixedParameter(name='aggregation', paramete

{'lr': 0.0001865322718407359, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0266 macroF1 0.0040
org:role: acc 0.5066
case:Project: acc 0.1114
case:Task: acc 0.0000
case:OrganizationalEntity: acc 0.0800
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0000
org:resource:role: acc 0.5066
time:timestamp: MAE 13.5511
case:RequestedAmount: MAE 1.5144
TOTAL_LOSS: 42.8624
epoch time 19.661094188690186s

Epoch: 1




VALIDATION
org:resource: acc 0.5066
Activity: acc 0.0824 macroF1 0.0371
org:role: acc 0.5066
case:Project: acc 0.1114
case:Task: acc 0.7074
case:OrganizationalEntity: acc 0.0800
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0000
org:resource:role: acc 0.5066
time:timestamp: MAE 9.4972
case:RequestedAmount: MAE 1.0597
TOTAL_LOSS: 36.1335
epoch time 19.011427640914917s

new best model found
Epoch: 2




VALIDATION
org:resource: acc 0.5103
Activity: acc 0.1256 macroF1 0.0857
org:role: acc 0.5066
case:Project: acc 0.1114
case:Task: acc 0.7074
case:OrganizationalEntity: acc 0.0800
case:Activity: acc 0.7047
case:RfpNumber: acc 0.0290
org:resource:role: acc 0.5254
time:timestamp: MAE 5.3015
case:RequestedAmount: MAE 1.0218
TOTAL_LOSS: 30.8060
epoch time 18.704867124557495s

new best model found
Epoch: 3



...


VALIDATION
org:resource: acc 0.9973
Activity: acc 0.6966 macroF1 0.4353
org:role: acc 0.9022
case:Project: acc 0.9858
case:Task: acc 0.7796
case:OrganizationalEntity: acc 0.9912
case:Activity: acc 1.0000
case:RfpNumber: acc 0.0006
org:resource:role: acc 0.9961
time:timestamp: MAE 0.5810
case:RequestedAmount: MAE 0.2574
TOTAL_LOSS: 12.8897
epoch time 14.200749397277832s

Epoch: 14




VALIDATION
org:resource: acc 0.9958
Activity: acc 0.6374 macroF1 0.4136
org:role: acc 0.8982
case:Project: acc 0.9882
case:Task: acc 0.7899
case:OrganizationalEntity: acc 0.9915
case:Activity: acc 1.0000
case:RfpNumber: acc 0.0000
org:resource:role: acc 0.9964
time:timestamp: MAE 0.6022
case:RequestedAmount: MAE 0.1590
TOTAL_LOSS: 12.7052
epoch time 16.764487743377686s

Epoch: 15




VALIDATION
org:resource: acc 0.9949
Activity: acc 0.6869 macroF1 0.4692
org:role: acc 0.8995
case:Project: acc 0.9885
case:Task: acc 0.7847
case:OrganizationalEntity: acc 0.9915
case:Activity: acc 1.0000
case:RfpNumber: acc 0.0021
org:resource:role: acc 0.9946
time:timestamp: MAE 0.2132
case:RequestedAmount: MAE 0.1556
TOTAL_LOSS: 12.6430
epoch time 14.257755279541016s

Validation performance didn't improve for 5 epochs. Training stops.


[INFO 01-13 20:29:53] ax.core.experiment: Attached data has some metrics ({'case:Task_acc', 'case:RfpNumber_acc', 'org:resource_acc', 'case:RequestedAmount_mae', 'MacroF1Act', 'Activity_acc', 'case:Project_acc', 'time:timestamp_mae', 'org:role_acc', 'case:Activity_acc', 'case:OrganizationalEntity_acc', 'Resource_to_role_acc', 'org:resource:role_acc'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[ERROR 01-13 20:29:53] ax.core.observation: Data contains metric Activity_acc that has not been added to the experiment. You can either update the `optimization_config` or attach it as a tracking metric using `Experiment.add_tracking_metrics` or `AxClient.add_tracking_metrics`. Ignoring all data for metric Activity_acc.
NoneType: None
[ERRO

{'lr': 0.013847456004837408, 'batch_size': 128, 'heads': 2, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
{'AVG_total_loss': 8.989108975250266}
Experiment(None)


In [32]:
from ax.service.utils.report_utils import exp_to_df

results = exp_to_df(experiment)

[WARNING 01-13 20:35:41] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [36]:
results.sort_values(by="MacroF1Act")

,trial_index,arm_name,trial_status,generation_method,AVG_total_loss,Activity_acc,MacroF1Act,Resource_to_role_acc,case:Activity_acc,case:OrganizationalEntity_acc,case:Project_acc,case:RequestedAmount_mae,case:RfpNumber_acc,case:Task_acc,org:resource:role_acc,org:resource_acc,org:role_acc,time:timestamp_mae,lr,batch_size,heads,hid,layers,aggregation
0,0,0_0,COMPLETED,Sobol,20.653662,0.688978,0.332212,0.879061,0.766202,0.204797,0.222402,0.010713,0.033084,0.705902,0.875659,0.879061,0.851846,0.044901,0.000187,256,1,128,2,max
4,4,4_0,COMPLETED,Sobol,17.025688,0.664739,0.356203,0.979078,0.998384,0.473210,0.361031,0.007214,0.033084,0.705902,0.984266,0.979078,0.897176,0.051361,0.001014,512,2,128,2,max
15,15,15_0,COMPLETED,BoTorch,10.499459,0.836877,0.528036,0.990815,0.999915,0.997704,0.997789,0.314560,0.026280,0.865964,0.992686,0.990815,0.930175,0.465339,0.020322,128,2,128,2,max
17,17,17_0,COMPLETED,BoTorch,10.408730,0.800391,0.549325,0.986988,0.999405,0.999320,0.997619,0.173452,0.000340,0.859500,0.986817,0.986988,0.935278,0.350159,0.100000,128,1,128,2,max
3,3,3_0,COMPLETED,Sobol,12.774159,0.844531,0.571823,0.991835,0.999150,0.952713,0.590066,0.048364,0.033254,0.709389,0.991325,0.991835,0.917078,0.056102,0.002055,256,2,128,2,max
10,10,10_0,COMPLETED,BoTorch,9.568845,0.882548,0.574514,0.995152,0.999235,0.998979,0.998299,0.338537,0.028661,0.908998,0.995578,0.995152,0.960963,0.322550,0.100000,128,2,128,2,max
14,14,14_0,COMPLETED,BoTorch,8.387796,0.855503,0.582776,0.995237,0.999915,0.999915,0.999235,0.084530,0.034189,0.931876,0.994132,0.995237,0.986477,0.104110,0.016517,128,2,128,2,max
8,8,8_0,COMPLETED,BoTorch,10.358970,0.819357,0.585990,0.994472,0.999490,0.998469,0.986222,0.310465,0.030022,0.836282,0.994897,0.994472,0.963344,0.524164,0.100000,512,2,128,2,max
9,9,9_0,COMPLETED,BoTorch,9.123206,0.871322,0.606028,0.994217,1.000000,0.998724,0.993706,0.351660,0.033764,0.894200,0.995578,0.994217,0.983416,0.136006,0.015101,256,2,128,2,max
16,16,16_0,COMPLETED,BoTorch,9.893248,0.920650,0.652874,0.995918,1.000000,1.000000,0.997619,0.154437,0.000255,0.823865,0.994812,0.995918,0.971424,0.115627,0.010826,128,1,128,2,max


In [34]:
results = results.sort_values(by="AVG_total_loss")

In [35]:
results.to_csv("BPI20R.csv", sep=",")